<a href="https://colab.research.google.com/github/maychr/NLP_AES_promt-based/blob/main/Prompt_essay_classifications_with_transformer_models/gpt_3_openAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation of the Environment
## Colab

In [1]:
# Installation of the following additional packages
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install openai


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 4.2 MB/s 
     |████████████████████████████████| 596 kB 40.2 MB/s 
     |████████████████████████████████| 6.5 MB 27.4 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 42 kB 716 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 162 kB 7.0 MB/s 
  Created wheel for openai: filename=openai-0.18.0-py3-none-any.whl size=52377 sha256=df28a7df49d929d790b740e2269fd14ea78bf9aa168fc54d03dfeddd472204f8
  Stored in directory: /root/.cache/pip/wheels/5f/f8/2d/eeca98adbe630fa9d3b34184d06791b38eebbb9622aaebdd47
Successfully built openai


# Data Import

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir("//content/gdrive/MyDrive/nlp_data/") 

import numpy
import pandas as pd
data = pd.read_csv("test_recoded_final.csv", sep= ";", encoding="UTF-8")


Mounted at /content/gdrive


In [3]:
# Casting the data into lists
label = data.iloc[0:, 1].to_list()
mail = data.iloc[0:, 0].to_list()


# Few-shot
## API and Preparation

In [4]:
# We call the Model via API
# It is Artificial Intelligence as a Service (AIaaS) and therefore a paid approach.

OPENAI_API_KEY='BLINDED'


In [5]:
# Defnition of labels and tokens to provide a logit bias.
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

labels = ['unhöflich', 'höflich']

def tokenize_labels(labels):
    """
    Converts a list of labels into a list of GPT-3 tokens.
    Adds preceding whitespace as needed in order to account for
    quirks in how GPT-3 handles tokenization.
    """    # Start with whitespace tokens
    tokens = [" ", "\n"]    # Tokenize each label by itself *and* with a preceding space.
    for label in labels:
        tokens += tokenizer.encode(label)
        tokens += tokenizer.encode(" " + label)

    return tokens
    

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [6]:
# Definition of logit bias.
def get_logit_bias(labels):
    """
    Returns a logit_bias that can be used to constrain GPT-3
    predictions to a set of pre-determined character sequences
    (i.e. phrases or words). Intended to be used for classification
    problems.
    """    
    tokens = tokenize_labels(labels)
    logit_bias: Dict[str, float] = {}
    for token in set(tokens):
        # Set the logit_bias for each token to 100, effectively
        # forcing GPT-3 to only choose from these tokens.
        logit_bias[str(token)] = 100    
        
    return logit_bias

logit_bias = get_logit_bias(labels)


## Run the Model

In [7]:
import os
import openai

openai.api_key = OPENAI_API_KEY

count = 0
output = []

runs = len(mail)

restart_sequence = "\n####"

# Run data in a loop
for count in range(len(mail)+1):
  if count == runs: 
    break

  # Define the prompt with examples and mails to classify
  few_shot = ("""Classify mails. Labels: unhöflich, höflich
  ####
  mail: Hallo Susanne, unter gegebenen Umständen kann ich die folgenden Aufgaben nicht bearbeiten. Die Excel-Simulation ist nicht dafür geeignet. Darüber hinaus funktioniert mein Taschenrechner nicht. Mit freundlichen Grüßen
  label: höflich
  #### 
  mail: Hallo Herr Neumann, ich empfehle einen Fremdbezug. Mit freundlichen Grüßen
  label: höflich
  ####
  mail: bitte kommen Sie rüber und erklären mir es selbst
  label: unhöflich
  ####
  mail: Guten Tag, leider kann ich auch diese Aufgabe nicht erledigen !!!!!!!!!!!!!!!!!!!!!!!
  label: unhöflich
  ####
  mail: Hallo Herr Neumann, können Sie nicht mal auch eine Aufgabe alleine lösen? 
  label: unhöflich
  ####
  mail: Hallo Patrick, bin leider überfordert, da dieses Programm es mir nicht leicht machts. Es kann weder etwas ausgedruckt noch mehrere Tabellen bzw. Mails gleichzeitig geöffnet werdern. So macht das keinen Sinn.
  label: unhöflich
  ####
  mail: Guden, Ich habe keinerlei Ahnung ;) Gruß der Boss! 
  label: unhöflich
  ####
  mail: Hallo Herr Neumann, ich habe heute so viele Aufgaben von Ihnen erhalten, dass ich hierzu nicht mehr komme. Im sorry bro :D
  label: unhöflich
  ####
  """,'\n',"mail: ",mail[count],'\n',"label:")

  few_shot = ''.join(few_shot)

  # Define output including several model controls
  response = openai.Completion.create(
  engine= "text-curie-001",
  prompt = few_shot,
  #logprobs = 3,
  logit_bias= {'14274': 100, '2704': 100, '289': 100, '403': 100, '488': 100, '71': 100, '9101': 100},
  temperature=0,
  max_tokens=4,
  top_p=.9,
  frequency_penalty=2,
  presence_penalty=2,
  stop=["####", "\n", "."])
  
  output.append(response.choices[0]['text'])

  count += 1

print(count, 'Out of loop')


627 Out of loop


## Further Processing of Model Output

In [8]:
label = output

In [9]:
pred_class = label

In [10]:
# Replace labels with 0 and 1 for evaulation
import pandas as pd
test_pred_class = pd.DataFrame(data=pred_class)

test_pred_class = test_pred_class.replace({' höflich':1, #' höflich\n':1, 'höflich\n ':1, 
                                           ' unhöflich':0}) #,'unhöfl':0})

In [11]:
# Get human labels for evaluation
test_label = data.iloc[:,1]

# Calculate Evaluation Metrics


In [12]:
 # Checking the test data results

from sklearn import metrics

test_label = test_label[0:runs]
test_pred_class = test_pred_class

# Mean accuracy
print("Mean Accuracy:\n", metrics.accuracy_score(test_label, test_pred_class))

# Confusion matrix
print("Confusion Matrix:\n", metrics.confusion_matrix(test_label, test_pred_class))

# F1 Score
print("F1 Score:\n", metrics.f1_score(test_label, test_pred_class))

# Precision
print("Precision:\n", metrics.precision_score(test_label, test_pred_class))

# Recall
print("Recall:\n", metrics.recall_score(test_label, test_pred_class))

# ROC AUC Score
print("ROC AUC:\n", metrics.roc_auc_score(test_label, test_pred_class))

# Cohen's Kappa Score
print("Cohen's Kappa:\n", metrics.cohen_kappa_score(test_label, test_pred_class))

# Total of incorrect predictions:
incorrects = test_label != test_pred_class[0]
print("Total of incorrect predictions:\n", sum(incorrects))


Mean Accuracy:
 0.9393939393939394
Confusion Matrix:
 [[ 20  30]
 [  8 569]]
F1 Score:
 0.9676870748299321
Precision:
 0.9499165275459098
Recall:
 0.9861351819757366
ROC AUC:
 0.6930675909878683
Cohen's Kappa:
 0.4832342862100377
Total of incorrect predictions:
 38


## Print Incorrect Predictions

We can further print incorrect predictions according to human gold-labels to

1.   Improve our few-shot prompt
2.   Check for human errors


In [ ]:
# Print incorrect classifications
test_text = data.iloc[:,0]

for input, prediction, label in zip(test_text, test_pred_class[0], test_label):
  if prediction != label:
    print(input, '\n\n', 'has been machine coded as', prediction, 'and should be', label, 'according to human coding.\n\n-------------------------------------------------------------------------------------\n')
